# Setup

In [35]:
import os
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

In [36]:
with open('secret/secret.txt') as f:
    api_key = f.readline().strip()

youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

# Load and Cache

In [37]:
def get_and_add_playlist_id_to_dataframe(dataframe, channel_ids):
    playlist_ids = []
    for channel_id in channel_ids:
        request = youtube.channels().list(
            part="contentDetails",
            id=channel_id
        )
        response = request.execute()
        playlist_ids.append(response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    dataframe['playlist_id'] = playlist_ids
    return dataframe

In [38]:
import ast

def get_video_data(video_data_path, channel_data):
    if os.path.exists(video_data_path):
        video_data = pd.read_csv(video_data_path)
    else:
        video_data = pd.DataFrame(columns=["channel_id","video_id", "video_title", "publish_date" ,"view_count", "like_count", "dislike_count", "comment_count"])
        video_ids = {}
        all_video_ids = ""
        try:
            for list_string,channel_id in zip(channel_data["Video_ID"],channel_data["channel_id"]):
                ids_list = ast.literal_eval(list_string)

                for id in ids_list:

                    video_ids[id] = channel_id

                    if all_video_ids == "":
                        all_video_ids += id
                    else:
                        all_video_ids += ',' + id

                    if len(all_video_ids) > 10:

                        video_request = youtube.videos().list(
                            part="snippet,statistics",
                            id=all_video_ids
                        )

                        video_response = video_request.execute()

                        for video in video_response["items"]:

                            try:
                                video_id = video["id"]
                            except KeyError:
                                video_id = float('nan')
                            try:
                                view_count = video["statistics"]["viewCount"]
                            except KeyError:
                                view_count = float('nan')
                            try:
                                like_count = video["statistics"]["likeCount"]
                            except KeyError:
                                like_count = float('nan')
                            try:
                                dislike_count = video["statistics"]["dislikeCount"]
                            except KeyError:
                                dislike_count = float('nan')
                            try:
                                comment_count = video["statistics"]["commentCount"]
                            except KeyError:
                                comment_count = float('nan')
                            try:
                                video_title = video["snippet"]["title"]
                            except KeyError:
                                video_title = float('nan')
                            try:
                                publish_date = video["snippet"]["publishedAt"]
                            except KeyError:
                                publish_date = float('nan')

                            row = pd.DataFrame({
                                "video_id": [video_id],
                                "channel_id": [video_ids[video_id]],
                                "video_title": [video_title],
                                "view_count": [view_count],
                                "like_count": [like_count],
                                "dislike_count": [dislike_count],
                                "comment_count": [comment_count],
                                "publish_date": [publish_date]
                            })

                            video_data = pd.concat([video_data, row], ignore_index=True)

                        all_video_ids = ""
                        video_ids = {}

        except Exception as e:
            print(f"Error {e}")
            video_response = {"items": []}

        

        if len(video_data) > 0:    
            video_data.to_csv(video_data_path, index=False)
    
    return video_data

In [39]:
channel_data_path = f"ChannelID_Data.csv"
video_data_path = f"video_data.csv"

channel_data =  pd.read_csv(channel_data_path)
video_data = pd.read_csv(video_data_path)
 

# Use Data

In [40]:
display(channel_data.head())
display(video_data.head())

,Unnamed: 0,channel_title,channel_id,description,view_count,subscriber_count,video_count,Category,playlist_id
0,0,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA
1,1,Music,UC-9-kyTW8ZkZNDHQJ6FgpwQ,Visit the YouTube Music Channel to find today’...,0,121000000,0,Music,UU-9-kyTW8ZkZNDHQJ6FgpwQ
2,2,Zee Music Company,UCFFbwnve3yF62-tVXkTyHqg,Zee Music Company is part of India's leading t...,69639694195,110000000,12150,Music,UUFFbwnve3yF62-tVXkTyHqg
3,3,BLACKPINK,UCOmHUn--16B90oW2L6FRR3A,BLACKPINK Official YouTube Channel\n블랙핑크 공식 유튜...,36756090437,94800000,603,Music,UUOmHUn--16B90oW2L6FRR3A
4,4,BANGTANTV,UCLkAepWjdylmXSltofFvsYQ,This is the official YouTube channel of BTS. 방...,23675450062,79000000,2606,Music,UULkAepWjdylmXSltofFvsYQ


,channel_id,video_id,video_title,publish_date,view_count,like_count,dislike_count,comment_count
0,UCq-Fj5jknLsUf-MWSy4_brA,BBAyRBTfsOU,"Vaaste Song: Dhvani Bhanushali, Tanishk Bagchi...",2019-04-06T11:56:37Z,1.613914e+09,13290142.0,NaN,363863.0
1,UCq-Fj5jknLsUf-MWSy4_brA,sCbbMZ-q4-I,"Lut Gaye (Full Song) Emraan Hashmi, Yukti | Ju...",2021-02-17T06:30:11Z,1.422724e+09,11124137.0,NaN,342825.0
2,UCq-Fj5jknLsUf-MWSy4_brA,JFcgOboQZ08,DILBAR Lyrical | Satyameva Jayate |John Abraha...,2018-07-09T13:50:00Z,1.338393e+09,5713883.0,NaN,190362.0
3,UCq-Fj5jknLsUf-MWSy4_brA,f6vY6tYvKGA,"Cham Cham Full Video | BAAGHI | Tiger Shroff, ...",2016-05-06T08:02:29Z,1.258221e+09,4196968.0,NaN,96179.0
4,UCq-Fj5jknLsUf-MWSy4_brA,NJ1NIIdHhXs,Bum Bum Bole (Full Song) Film - Taare Zameen P...,2011-05-24T20:39:36Z,1.162779e+09,3122527.0,NaN,70381.0


In [41]:
channel_data = channel_data.drop(columns=['Unnamed: 0'])
video_data = video_data.drop(columns=['dislike_count'])

# merge by channel_id
merged_data = pd.merge(channel_data, video_data, on="channel_id")
display(merged_data.head())

,channel_title,channel_id,description,view_count_x,subscriber_count,video_count,Category,playlist_id,video_id,video_title,publish_date,view_count_y,like_count,comment_count
0,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,BBAyRBTfsOU,"Vaaste Song: Dhvani Bhanushali, Tanishk Bagchi...",2019-04-06T11:56:37Z,1.613914e+09,13290142.0,363863.0
1,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,sCbbMZ-q4-I,"Lut Gaye (Full Song) Emraan Hashmi, Yukti | Ju...",2021-02-17T06:30:11Z,1.422724e+09,11124137.0,342825.0
2,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,JFcgOboQZ08,DILBAR Lyrical | Satyameva Jayate |John Abraha...,2018-07-09T13:50:00Z,1.338393e+09,5713883.0,190362.0
3,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,f6vY6tYvKGA,"Cham Cham Full Video | BAAGHI | Tiger Shroff, ...",2016-05-06T08:02:29Z,1.258221e+09,4196968.0,96179.0
4,T-Series,UCq-Fj5jknLsUf-MWSy4_brA,"""Music can change the world"". T-Series is Indi...",267112024995,274000000,21739,Music,UUq-Fj5jknLsUf-MWSy4_brA,NJ1NIIdHhXs,Bum Bum Bole (Full Song) Film - Taare Zameen P...,2011-05-24T20:39:36Z,1.162779e+09,3122527.0,70381.0
